TRADE HISTORY LENGTH 2, CORRECTLY ORDERED 

YIELD CURVE HISTORY 5MIN_12_AVERAGE , CORRECTLY ORDERED 

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import time
import gc

import numpy as np
from google.cloud import bigquery
from google.cloud import storage

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from pandas.tseries.offsets import BDay

from tensorflow.keras.layers import Embedding
from tensorflow.keras import activations
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import matplotlib.pyplot as plt
import pickle5 as pickle


from ficc.utils.nelson_siegel_model import *
from ficc.utils.diff_in_days import *
from ficc.utils.auxiliary_functions import sqltodf


from IPython.display import display, HTML
import os


from ficc.data.process_data import process_data
from ficc.utils.auxiliary_variables import PREDICTORS, NON_CAT_FEATURES, BINARY, CATEGORICAL_FEATURES, IDENTIFIERS, PURPOSE_CLASS_DICT, NUM_OF_DAYS_IN_YEAR
from ficc.utils.gcp_storage_functions import upload_data, download_data
from ficc.utils.auxiliary_variables import RELATED_TRADE_BINARY_FEATURES, RELATED_TRADE_NON_CAT_FEATURES, RELATED_TRADE_CATEGORICAL_FEATURES

import sys
sys.path.append('../../')
from ficc_keras_utils import *
import ficc_keras_utils

pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(f'TF Version: {tf.__version__}')

2023-05-18 20:34:31.701112: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-18 20:34:31.714075: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-18 20:34:31.715630: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Initializing pandarallel with 16.0 cores
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
TF Version: 2.7.0


In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jupyter/ficc/isaac_creds.json"
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.options.mode.chained_assignment = None

bq_client = bigquery.Client()
storage_client = storage.Client()

##COMMON VARIABLES
#DATA WINDOW
train_start = ficc_keras_utils.train_start
train_end = ficc_keras_utils.train_end
test_start = ficc_keras_utils.test_start
test_end = ficc_keras_utils.test_end
#MODEL PARAMETERS 
VALIDATION_SPLIT = ficc_keras_utils.VALIDATION_SPLIT
LEARNING_RATE = ficc_keras_utils.LEARNING_RATE
BATCH_SIZE = ficc_keras_utils.BATCH_SIZE
NUM_EPOCHS = 150 #ficc_keras_utils.NUM_EPOCHS
DROPOUT = ficc_keras_utils.DROPOUT

##NOTEBOOK SPECIFIC VARIABLES 
TRADE_SEQUENCE_LENGTH = 2
YIELD_SEQUENCE_LENGTH = 11
NUM_FEATURES = 6
# target_variable = 'new_ys_diff'
target_variable = 'new_ys'

In [ ]:
print(train_start ,
train_end ,
test_start ,
test_end)

In [4]:
processed_data = pd.read_pickle('../../processed_data-2023-05-12.pkl')

In [ ]:
additional_features = ['ttypes', 'diff_size', 'abs_last_yield_spread', 'abs_diff_size', 'days_duration']
YS_COLS = ['max_ys_ys', 'max_ys_ttypes', 'max_ys_ago', 'max_ys_qdiff', 'min_ys_ys', 'min_ys_ttypes', 'min_ys_ago', 'min_ys_qdiff', 'max_qty_ys', 'max_qty_ttypes', \
           'max_qty_ago', 'max_qty_qdiff', 'min_ago_ys', 'min_ago_ttypes', 'min_ago_ago', 'min_ago_qdiff', 'D_min_ago_ys', 'D_min_ago_ttypes', 'D_min_ago_ago', 'D_min_ago_qdiff',\
           'P_min_ago_ys', 'P_min_ago_ttypes', 'P_min_ago_ago', 'P_min_ago_qdiff', 'S_min_ago_ys', 'S_min_ago_ttypes', 'S_min_ago_ago', 'S_min_ago_qdiff']
encoders = {}
fmax = {}
for f in CATEGORICAL_FEATURES:
    print(f)
    if f in ['rating', 'incorporated_state_code', 'trade_type', 'purpose_class']:
        fprep = preprocessing.LabelEncoder().fit(categorical_feature_values[f])
    else:
        fprep = preprocessing.LabelEncoder().fit(processed_data[f].drop_duplicates())
    fmax[f] = np.max(fprep.transform(fprep.classes_))
    encoders[f] = fprep
    
with open('encoders.pkl','wb') as file:
    pickle.dump(encoders,file)
    
for col in YS_COLS:
    if 'ttypes' in col and col not in PREDICTORS:
        PREDICTORS.append(col)
        CATEGORICAL_FEATURES.append(col)
    elif col not in PREDICTORS:
        NON_CAT_FEATURES.append(col)
        PREDICTORS.append(col)
        
if 'target_attention_features' not in PREDICTORS:
    PREDICTORS.append('target_attention_features')
    
if 'ficc_treasury_spread' not in PREDICTORS:
    PREDICTORS.append('ficc_treasury_spread')
    NON_CAT_FEATURES.append('ficc_treasury_spread')
    
for col in ['new_ficc_ycl', 'new_real_time_ficc_ycl']:     
    if col not in PREDICTORS:
        PREDICTORS.append(col)
        NON_CAT_FEATURES.append(col)

In [7]:
%%time

processed_data['yield_curve_history_5min_12_averaged_first_differenced'] = processed_data['yield_curve_history_5min_12_averaged']\
.apply(lambda x: -np.diff(x))\
.apply(lambda x: x[::-1])

CPU times: user 29 s, sys: 486 ms, total: 29.5 s
Wall time: 29.5 s


In [8]:
%%time

processed_data['trade_history_shortened'] = processed_data['trade_history'].apply(lambda x: x[:TRADE_SEQUENCE_LENGTH])

CPU times: user 11 s, sys: 498 ms, total: 11.5 s
Wall time: 11.4 s


In [9]:
%%time

processed_data['trade_history_fixed'] = processed_data['trade_history_shortened'].apply(lambda x: x[::-1, :])

CPU times: user 10.8 s, sys: 329 ms, total: 11.1 s
Wall time: 11.1 s


Add Overnight Movements

In [10]:
from ficc.yield_curve.YieldCurve import *
from ficc.yield_curve.time_series_functions import *
from ficc.yield_curve.auxiliary_functions import *

In [12]:
rtyc = RealTimeYieldCurve(start_date='2022-12-25', end_date='2023-04-01')
rtyc.initialize_data()


Setting start date to 2022-12-25 00:00:00-05:00
Setting end date to 2023-04-01 00:00:00-04:00
Loading real time yield curve values from BigQuery
Real time yield curve values loaded, data spans 2022-12-28 09:30:00 to 2023-03-31 15:59:00
Estimating yield curves for maturity T = 0.1 to T = 30.0
Estimating and saving yield curves for maturity T = 0.1 to T = 30.0


In [39]:
rtyc_values = rtyc.yield_curves.copy()
overnight_moves = between_day_overnight_move(rtyc_values)
overnight_moves.index = pd.to_datetime(overnight_moves.index)
overnight_moves_dict = overnight_moves[key_maturities].loc[train_start:].T.to_dict()

In [ ]:
train_filter = (processed_data.trade_date < train_end) & (processed_data.trade_date >= train_start)
test_filter = (processed_data.trade_date >= test_start) & (processed_data.trade_date <test_end)
                                                            
train_dataframe = processed_data[train_filter]\
.sort_values(by='trade_date', ascending=True)\
.reset_index(drop=True)

test_dataframe = processed_data[test_filter]\
.sort_values(by='trade_date', ascending=True)\
.reset_index(drop=True)

print('Training data start: {}, end: {}'.format(train_dataframe.trade_date.min(),train_dataframe.trade_date.max()))
print('Test data start: {}, end: {}'.format(test_dataframe.trade_date.min(),test_dataframe.trade_date.max()))

In [78]:
def return_overnight_cols(trade_dates):
    default = dict(zip(range(6), np.zeros(6)))
    return pd.DataFrame(trade_dates.apply(lambda x: overnight_moves_dict.get(x, default).values()).to_list())

In [86]:
%%time

overnight_cols_train = return_overnight_cols(train_dataframe.trade_date)

CPU times: user 10.4 s, sys: 49.6 ms, total: 10.4 s
Wall time: 10.4 s


In [87]:
%%time

overnight_cols_test = return_overnight_cols(test_dataframe.trade_date)

CPU times: user 4.8 s, sys: 2.32 ms, total: 4.81 s
Wall time: 4.81 s


In [91]:
overnight_col_names = ['ficc_ycl_overnight_'+str(x) for x in key_maturities]
train_dataframe[overnight_col_names] = overnight_cols_train
test_dataframe[overnight_col_names] = overnight_cols_test

In [96]:
for col in overnight_col_names:
    if col not in PREDICTORS:
        PREDICTORS.append(col) 
    if col not in NON_CAT_FEATURES:
        NON_CAT_FEATURES.append(col)

Using Charles'LightGBM Implementation:

In [8]:
for col in CATEGORICAL_FEATURES + BINARY:
    train_dataframe[col] = train_dataframe[col].astype('category')
    test_dataframe[col] = test_dataframe[col].astype('category')

NameError: name 'train_dataframe' is not defined

In [ ]:
from sklearn.ensemble import VotingRegressor
import lightgbm
from lightgbm import LGBMRegressor

defaultdepth = 8
defaultseed = 881
defaultloss = 'mae' # 'huber' # 'fair'
huberdelta = 30
default_n_jobs = 4

def gbmprep(df, only=[], plus=[], minus=[]):
    if only != []: 
        USECOLS = only
    else:
        USECOLS = df.select_dtypes(include=['category','bool','number']).columns
    USECOLS = ( set(USECOLS) - mkleakers(TARGETS) - TOOMANY | set(plus) ) - set(minus)
    return df[ list(USECOLS) ]


def myLGBM(seed = defaultseed, depth = defaultdepth, loss = defaultloss):
    return LGBMRegressor(max_depth=depth, num_leaves=depth*10,  objective=loss, verbosity=-1, alpha = huberdelta,
                         n_estimators=depth*30, subsample = 0.5, subsample_freq = 10, # linear_tree = True,
                         random_state = seed, n_jobs = default_n_jobs, device_type = "cpu") 

def mkensemble(n = 4, seed = defaultseed, depth = defaultdepth, loss = defaultloss):
    regressors = []
    for j in range(0,n):
        regressors = regressors + [( 'm'+str(j), myLGBM(seed+j, depth, loss) )]
    return VotingRegressor( regressors, n_jobs = default_n_jobs, verbose = False )

# def trtestrows(df,target):
#     global first_train_date, first_test_date
#     train_rows = (df.trade_date >= first_train_date) & (df.trade_date < first_test_date) & df[target].notnull()
#     test_rows = (df.trade_date >= first_test_date) & df[target].notnull()
#     return train_rows, test_rows
    
def ess(weights):
    return weights.sum()**2 / (weights**2).sum()

def traintest(train_dataframe, test_dataframe, target, n = 4, seed = defaultseed, depth = defaultdepth, loss = defaultloss, sample_weight=None,
              only=[], plus=[], minus=[], showplot=5, evaltrain=False):
    global TARGETS
    TARGETS = TARGETS | { target }
    
    # train_rows, test_rows = trtestrows(df,target)

    ens = mkensemble(n, seed, depth, loss)
    if n == 1:
        text = "Training one model "
    else:
        text = f"Training {n} models" 
    print(text + f"with {len(mydf.columns)} columns on {np.sum(train_rows)} examples starting {first_train_date:%Y-%m-%d}")
    if sample_weight is not None:
        print( f"Weighted samples with effective sample size = {ess(sample_weight):8.0f}" )
    print(f"Evaluating {target} predictions on {np.sum(test_rows)} examples starting {first_test_date:%Y-%m-%d}")
    ens.fit( train_dataframe[PREDICTORS], train_dataframe[target], sample_weight )
    uss()

    if evaltrain:
        maeval(train_dataframe, ens, target, only, plus, minus)
    maeval(test_dataframe, ens, target, only, plus, minus)
    drawpoints(df, test_rows, target, showplot)     
    return ens

def extend(df, name, vals):
    # assert len(df) == len(rows), "*** len(df) != len(rows)"
    # if len(vals) == len(df): 
    #     vals = vals[rows]
    # assert len(vals) == np.sum(rows), "*** len(vals) != np.sum(rows)"
    
    if not name in df.columns: df[name] = 0.0
    df.loc[:, name] = vals
    return

def maeval(df, model, label, only=[], plus=[], minus=[]):
    dfp = model.predict(df[PREDICTORS])
    extend(df, label + "_preds",  dfp )

    delta = df[label] - dfp
    extend(df, label + "_err", delta )
    
    da = delta.abs()
    extend(df, label + "_ae", da )

    n = len(da)
    base = f"\n{label} n = {n}  bias = {delta.mean():5.2f}  MAE={da.mean():5.2f} +/- {da.std():.2f}"
    print( base + f" ({da.std()/np.sqrt(n):.2f})\t median {da.median():5.2f}" )
    return df


def myplotimportance(model, nfeatures = 30):
    if isinstance( model, LGBMRegressor ): 
        m = model
    elif isinstance( model.estimators_[0], LGBMRegressor ): 
        m = model.estimators_[-1]
    else: 
        print("*** not a valid model")
    lightgbm.plot_importance(m, importance_type="gain", precision=0, ignore_zero=False,\
                             max_num_features=nfeatures, figsize=(8,np.ceil(nfeatures/7)) )
    
def imptdf(model):
    m = model.estimators_[-1]
    imps = pd.DataFrame([m.feature_name_, m.booster_.feature_importance(importance_type='gain'), m.feature_importances_]).T
    imps.columns = ['name', 'gain', 'splits']
    imps = imps.sort_values(by='gain', ascending=False)
    # imps['gain'] = (imps.gain/imps.gain.mean()) / (imps.splits/imps.splits.mean())
    imps['gain'] = imps.gain/1e6
    return imps

In [ ]:
TARGET = 'new_ys'
%time nysmodel = traintest(data, TARGET, n = 1)
myplotimportance(nysmodel,30)